In [ ]:
def bar_chart_improvement(df, athlete_id, metric_dict, unit_dict, pdf):
    """
    Generate a bar chart showing the most and least improved metrics for an athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): Athlete ID.
        metric_dict (dict): Dictionary of metrics to analyze.
        unit_dict (dict): Dictionary mapping metrics to their units.
        pdf (PdfPages): PDF object to save the plot.

    Returns:
        None
    """
    # Retrieve percentiles and athlete information based on current level
    percentiles, athlete_name, athlete_level = calculate_percentiles(df, athlete_id, metric_dict)

    # Create metrics summary and calculate improvement
    metrics_summary = pd.DataFrame(percentiles).T
    metrics_summary['Improvement'] = metrics_summary['recent_percentile'] - metrics_summary['previous_percentile']

    # Sort and reverse the order to have most improved at the top
    combined_metrics = metrics_summary.sort_values(by='Improvement', ascending=False).iloc[::-1]

    # Generate colors for the bars
    cmap = cm.get_cmap('coolwarm')
    colors = cmap(np.linspace(0, 1, len(combined_metrics)))

    fig, ax = plt.subplots(figsize=(14, 10))

    for i, (metric, row) in enumerate(combined_metrics.iterrows()):
        # Get units and define color
        metric_parts = metric.split(' - ')
        units = unit_dict.get(tuple(metric_parts[:2]), '')  # Extract test type and sub-type for unit lookup
        color = colors[i]

        # Plot bars for recent and previous percentiles
        ax.barh(metric, row['recent_percentile'], color=color, alpha=0.7)
        ax.barh(metric, row['previous_percentile'], color='lightgray', alpha=0.5)

        # Add metric value annotation
        if pd.notna(row['recent_value']):
            ax.text(row['recent_percentile'] + 1, i, f"{row['recent_value']:.1f} {units}",
                    va='center', ha='left', fontsize=8, color='black')

        # Position the improvement annotation conditionally based on positive or negative improvement
        improvement_text = f"{row['Improvement']:+.1f}"
        if row['Improvement'] < 0:
            # Position to the left of the bar for negative improvement
            ax.text(row['previous_percentile'] - 1, i, improvement_text, va='center', ha='right',
                    fontsize=8, color='white')
        else:
            # Centered over the bar for positive improvement
            ax.text((row['previous_percentile'] + row['recent_percentile']) / 2, i,
                    improvement_text, va='center', ha='center', fontsize=8, color='white')

    # Finalize plot details
    ax.set_xlim(0, 100)
    ax.set_xlabel("Percentile Rank")
    ax.set_title(f"Most and Least Improved Metrics\nLevel: {athlete_level}", fontweight='bold')
    ax.grid(axis='x', color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
    plt.tight_layout()

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)